# 03 함수적 자료구조

* 스칼라로 배우는 함수형 프로그래밍 - 3장
* 김무성

# 차례

* 3.1 함수적 자료구조의 정의
* 3.2 패턴 부합
* 3.3 함수적 자료구조의 자료 공유
    - 3.3.1 자료 공유의 효율성
    - 3.3.2 고차 함수를 위한 형식 추론 개선
* 3.4 목록에 대한 재귀와 고차 함수로의 일반화
    - 3.4.1 그 외의 목록 조작 함수들
    - 3.4.2 단순 구성요소들로 목록 함수를 조립할 때의 효율성 손실
* 3.5 트리

# 3.1 함수적 자료구조의 정의

* 함수적 자료구조란 오직 순수 함수만으로 조작되는 자료구조다.
* 함수적 자료구조는 불변이(immutable)이다.
* 이러면 여분의 복사가 많이 일어나지 않을까? (자료구조끼리 더하면 합쳐진 새로운 자료구조를 반환하는데) -> "그렇지 않다" - 왜? 자료공유 방식을 사용하므로.

## 단일 연결 목록 예제(Single Linked List) 

#### 스칼라 문법 참고

* trait : 추상 인터페이스
* case 
* object  
* 형식매개변수 : [A]
* 공변(covariant) 매개변수임을 뜻하는 가변 지정자(variance annotation) : +
* 동반 객체(companion object) : 동반 객체는 자료 형식가 같은 이름의 object로, 자료 형식의 값들을 생성하거나 조작하는 여러 편의용 함수들을 담는 목적으로 쓰인다. 

<img src="figures/sll.png" width=600 />

In [164]:
sealed trait List[+A] 
case object Nil extends List[Nothing]
case class Cons[+A](head: A, tail: List[A]) extends List[A]

object List {
    def sum(ints: List[Int]) : Int = ints match {
        case Nil => 0
        case Cons(x,xs) => x + sum(xs)
    }
    
    def product(ds: List[Double]) : Double = ds match {
        case Nil => 1.0
        case Cons(0.0, _) => 0.0
        case Cons(x, xs) => x * product(xs)
    }
    
    def apply[A](as: A*): List[A] = {
        if (as.isEmpty) Nil
        else Cons(as.head, apply(as.tail: _*))
    }

}

<img src="figures/list3.1.png" width=600 />

### 자료 생성자의 사용 예시

In [112]:
val ex1: List[Double] = Nil

Nil

In [113]:
val ex2: List[Int] = Cons(1, Nil)

Cons(1,Nil)

In [114]:
val ex3: List[String] = Cons("a", Cons("b", Nil))

Cons(a,Cons(b,Nil))

# 3.2 패턴 부합

#### 스칼라 문법 참고

* 동반 객체(companion object) : 동반 객체는 자료 형식가 같은 이름의 object로, 자료 형식의 값들을 생성하거나 조작하는 여러 편의용 함수들을 담는 목적으로 쓰인다. 
* 패턴 부합 구문 :
    - 표현식(대상target) match { case 패턴 => 결과 }
    - 예) 
        - List(1,2,3) match { case Cons(h,_) => h} 
        - 결과는 1 

In [165]:
def sum(ints: List[Int]) : Int = ints match {
    case Nil => 0
    case Cons(x,xs) => x + sum(xs)
}
    
def product(ds: List[Double]) : Double = ds match {
    case Nil => 1.0
    case Cons(0.0, _) => 0.0
    case Cons(x, xs) => x * product(xs)
}

<img src="figures/ml.png" width=600 />

In [120]:
List(1,2,3) match { case _ => 42}

42

In [121]:
List(1,2,3) match { case Cons(h,_) => h }

1

In [122]:
List(1,2,3) match { case Cons(_,t) => t }

Cons(2,Cons(3,Nil))

In [25]:
// 실행시점 MatchError 오류
// MatchError는 부합 표현식의 경우 문 중 대상과 부합하는 것이 하나도 없음을 뜻함
// 패텬이 표현식과 부합하는지 판정하는 규칙 
//  - 변수는 모든 것에 부합
//  - 자료 생성자는 해당 형태의 값에만 부합
//  - 패턴으로소 Nil은 오로지 Nil 값에만 부합, 패턴으로서 Cons(h,t)나 Cons(x,xs)는 오직 Cons 값들에만 부합
List(1,2,3) match { case Nil => 42 }

: 

### 연습문제 3.1

다음 패턴 부합 표현식의 결과는 무엇인가?

In [123]:
val x = List(1,2,3,4,5) match {
    case Cons(x, Cons(2, Cons(4,_))) => x
    case Nil => 42
    case Cons(x, Cons(y, Cons(3, Cons(4,_)))) => x + y
    case Cons(h,t) => h + sum(t)
    case _ => 101
}

3

# 3.3 함수적 자료구조의 자료 공유

* 자료가 불변이라면, 목록에 요소를 추가하거나 제거하는 함수는 어떻게 작성해야 할까?
* 기존 목록(xs)의 앞에 1이라는 요소를 추가하려면 Cons(1,xs)라는 새 목록을 만들면 된다. 
    - 목록은 불변이므로, xs를 복사할 필요가 없다. 재사용하면 된다.
* 이렇게 기존 요소가 재사용되는 것을 자료 공유(data sharing)이라고 한다.
* 기존 목록 mylist = Cons(x, xs)의 앞(첫) 요소를 제거하려면 그냥 xs을 돌려주면 된다.
    - 실질적인 제거는 일어나지 않는다.
* 원래의 목록은 여전히 사용가능한 상태이다. - 이를 두고, 함수적 자료구조는 영속적(persistent)이라고 말한다.

<img src="figures/share.png" width=600 />

### 연습문제 3.2

* List의 첫 요소를 제거하는 함수 tail을 구현하라. 이 함수가 상수 시간으로 실행됨을 주의할 것. Nil인 List도 지원하도록 독자의 구현을 수정하는 여러 가지 방법들도 고려해 보라. 이에 대해서는 다음 장에서 좀 더 살펴볼 것이다.

In [124]:
def tail[A](l: List[A]): List[A] = 
  l match {
    case Nil => sys.error("tail of empty list")
    case Cons(_,t) => t
  }

In [125]:
val x = List(1,2,3,4,5)
tail(x)

Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Nil)))))

### 연습문제 3.3

* 같은 맥락에서, List의 첫 요소를 다른 값으로 대체하는 함수 setHead를 구현하라.

In [ ]:
// input your code

## 3.3.1 자료 공유의 효율성

### 연습문제 3.4

* tail을 일반화해서, 목록에서 처음 n개의 요소를 제거하는 함수 drop을 구현하라. 이 함수의 실행시간은 제거되는 원소의 개수에만 비례함을 주의할 것. List 전체의 복사본을 만들 필요는 없다.

In [156]:
def drop[A](l: List[A], n: Int): List[A] = 
  if (n <= 0) l 
  else l match {
    case Nil => Nil
    case Cons(_,t) => drop(t, n-1) 
  }

In [160]:
drop(List(1,2,3,4,5), 2)

Cons(3,Cons(4,Cons(5,Nil)))

### 연습문제 3.5

* 주어진 술어(predicate)와 부합하는 List의 앞 요소들(prefix)을 제거하는 함수 dropWhile을 구현하라

In [174]:
def dropWhile[A](l: List[A], f: A => Boolean): List[A] = 
  l match {
    case Cons(h,t) if f(h) => dropWhile(t, f) 
    case _ => l
  }

In [162]:
dropWhile(List(1,2,3,4,5), (x:Int) => x <4)

Cons(4,Cons(5,Nil))

### 자료 공유의 놀라운 예

다음 함수는 한 목록의 요소를 다른 목록의 끝에 추가한다.

In [31]:
// 오직 첫 목록이 다 소진될 때까지만 값들을 복사한다
// 이 함수의 실행 시간과 메모리 사용량은 오직 a1의 길이에만 의존한다.
// 목록의 나머지는 그냥 a2를 가리킬 뿐이다.
// 불편이 연결 목록이 배열보다 훨씬 효율적이다.
def append[A](a1: List[A], a2: List[A]): List[A] = 
    a1 match {
        case Nil => a2
        case Cons(h, t) => Cons(h, append(t, a2))
    }

### 연습문제 3.6

* 그러나 모든 것이 효율적이지는 않다. 한 List의 마지막 요소를 제외한 모든 요소로 이루어진 List를 돌려주는 함수 init을 구현하라.
* 예를 들어 List(1,2,3,4)에 대해 init은 List(1,2,3)을 돌려주어야 한다.
* 이 함수를 tail처럼 상수 시간으로 구현할 수 없는 이유는 무엇인가?

In [ ]:
def init[A](l: List[A]): List[A]

In [ ]:
// Enter your Answer

## 3.3.2 고차 함수를 위한 형식 추론 개선

In [ ]:
def dropWhile[A](l: List[A], f: A => Boolean): List[A]

In [172]:
val xs: List[Int] = List(1,2,3,4,5)

Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Nil)))))

In [176]:
val ex1 = dropWhile(xs, (x: Int) => x < 4)

Cons(4,Cons(5,Nil))

In [178]:
def dropWhile[A](as: List[A])(f: A => Boolean): List[A] = 
  as match {
    case Cons(h,t) if f(h) => dropWhile(t)(f) 
    case _ => as
  }

In [179]:
val ex1 = dropWhile(xs)(x => x < 4)

Cons(4,Cons(5,Nil))

# 3.4 목록에 대한 재귀와 고차 함수로의 일반화

## 3.4.1 그 외의 목록 조작 함수들

## 3.4.2 단순 구성요소들로 목록 함수를 조립할 때의 효율성 손실

# 3.5 트리

# 참고자료

* [1] 스칼라로 배우는 함수형 프로그래밍 - http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791185890180
* [2] 책 예제 코드 github - https://github.com/fpinscala/fpinscala